
# 숙명여자대학교 기계시스템학부 딥러닝 2025: Lab6

## Topic: **Synthetic UI Element Detection Dataset Generator**
## (강사: 심주용)

Creat a synthetic dataset for training object detection models to recognize user interface (UI) components, specifically **icons** and **panels**.

## Features

- **Verified Public Icons**: Downloads a curated list of public icons from [Icons8](https://icons8.com/).
- **Realistic Layouts**: Generates panels that are either vertically or horizontally elongated.
- **Aligned Icons**: Icons are placed in a single row or column aligned with the orientation of each panel.
- **Dynamic Panel Sizing**: Each panel’s size adjusts based on the number and size of icons it contains.
- **Non-Overlapping Panels**: Ensures that no two panels overlap in a single image.
- **Object Detection Annotations**: Outputs labels in YOLO format (`class x_center y_center width height`) for both icons (class `0`) and panels (class `1`).

## Output

- `images/`: Contains generated synthetic UI images.
- `labels/`: Contains corresponding YOLO-format annotation `.txt` files.

This setup is ideal for creating training data for UI automation, accessibility tools, or mobile app design analysis.


# Generate Train Data

In [23]:
import os
import requests
from PIL import Image, ImageDraw, UnidentifiedImageError
from io import BytesIO
import random

# Define directories
os.makedirs('datasets', exist_ok=True)
images_path = 'datasets/icon/images/train'
labels_path = 'datasets/icon/labels/train'
os.makedirs(images_path, exist_ok=True)
os.makedirs(labels_path, exist_ok=True)

# Verified list of public icon URLs
icons_base_url = "https://img.icons8.com/color/96/000000/"
icon_names = [
    "home--v1", "settings--v1", "search--v1", "user-male-circle--v1", "calendar--v1", "camera--v1", "phone--v1", "lock--v1", "cloud--v1",
    "facebook", "twitter", "instagram", "linkedin", "whatsapp", "youtube", "github", "google-logo", "dropbox", "spotify",
    "paypal", "visa", "mastercard", "bitcoin", "alarm", "windows-10", "android-os", "ubuntu", "linux",
    "trash", "download", "upload", "edit", "copy", "paste", "print", "refresh", "save",
    "bookmark", "shopping-cart", "tag", "star", "bell", "document", "briefcase", "key", "map",
    "clock", "gift", "graph", "chat", "network", "shield", "lightning-bolt", "rocket",
    "puzzle", "trophy", "globe", "flag", "compass", "paper-plane", "thumbs-down", "play", "pause",
    "stop", "rewind", "forward", "microphone", "headphones", "speaker", "video",
    "music", "film-reel", "paint-palette", "scissors", "hammer", "wrench", "gear", "car", "bus",
    "train", "bicycle", "motorcycle", "flower", "sun", "moon", "snowflake"
]
icons = [f"{icons_base_url}{name}.png" for name in icon_names]

# Cache downloaded icons to avoid repeated requests
icon_cache = {}

def download_icon(url):
    if url not in icon_cache:
        response = requests.get(url)
        try:
            icon_cache[url] = Image.open(BytesIO(response.content)).convert("RGBA")
        except UnidentifiedImageError:
            print(f"Failed to load image from URL: {url}")
            icon_cache[url] = None
    return icon_cache[url]

def is_overlapping(box1, box2):
    return not (box1[2] <= box2[0] or box1[0] >= box2[2] or box1[3] <= box2[1] or box1[1] >= box2[3])

def generate_synthetic_image(img_idx):
    panel_width, panel_height = 800, 470
    panel_color = tuple(random.randint(150, 255) for _ in range(3))
    panel = Image.new("RGB", (panel_width, panel_height), panel_color)
    draw = ImageDraw.Draw(panel)

    num_panels = random.randint(3, 10)
    annotations = []
    existing_boxes = []

    attempts = 0
    while len(existing_boxes) < num_panels and attempts < 50:
        arrange_vertical = random.choice([True, False])
        num_icons = random.randint(2, 6)
        icon_spacing = 10
        icon_size = random.randint(40, 60)

        if arrange_vertical:
            panel_w = icon_size + 20
            panel_h = num_icons * icon_size + (num_icons - 1) * icon_spacing + 20
        else:
            panel_w = num_icons * icon_size + (num_icons - 1) * icon_spacing + 20
            panel_h = icon_size + 20

        panel_x = random.randint(0, panel_width - panel_w)
        panel_y = random.randint(0, panel_height - panel_h)
        panel_box = [panel_x, panel_y, panel_x + panel_w, panel_y + panel_h]

        if any(is_overlapping(panel_box, eb) for eb in existing_boxes):
            attempts += 1
            continue

        existing_boxes.append(panel_box)
        panel_color_inner = tuple(random.randint(100, 200) for _ in range(3))
        draw.rectangle(panel_box, fill=panel_color_inner)

        x_center = (panel_x + panel_w / 2) / panel_width
        y_center = (panel_y + panel_h / 2) / panel_height
        width = panel_w / panel_width
        height = panel_h / panel_height
        annotations.append(f"1 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

        for i in range(num_icons):
            icon_url = random.choice(icons)
            icon = download_icon(icon_url)

            if icon is None:
                continue

            icon_resized = icon.resize((icon_size, icon_size))

            if arrange_vertical:
                icon_x = panel_x + (panel_w - icon_size) // 2
                icon_y = panel_y + 10 + i * (icon_size + icon_spacing)
            else:
                icon_x = panel_x + 10 + i * (icon_size + icon_spacing)
                icon_y = panel_y + (panel_h - icon_size) // 2

            panel.paste(icon_resized, (icon_x, icon_y), icon_resized)

            x_center_icon = (icon_x + icon_size / 2) / panel_width
            y_center_icon = (icon_y + icon_size / 2) / panel_height
            width_icon = icon_size / panel_width
            height_icon = icon_size / panel_height
            annotations.append(f"0 {x_center_icon:.6f} {y_center_icon:.6f} {width_icon:.6f} {height_icon:.6f}")

        attempts += 1

    img_path = os.path.join(images_path,f'synthetic_{img_idx}.png')
    panel.save(img_path)

    annotation_path = os.path.join(labels_path,f'synthetic_{img_idx}.txt')
    with open(annotation_path, 'w') as f:
        for annotation in annotations:
            f.write(annotation + "\n")

for i in range(100):
    generate_synthetic_image(i)

print("Synthetic UI dataset generated with non-overlapping panels and aligned icons!")

Synthetic UI dataset generated with non-overlapping panels and aligned icons!


# Generate Validation Data

In [21]:
import os
import requests
from PIL import Image, ImageDraw, UnidentifiedImageError
from io import BytesIO
import random

# Define directories
os.makedirs('datasets', exist_ok=True)
images_path = 'datasets/icon/images/val'
labels_path = 'datasets/icon/labels/val'
os.makedirs(images_path, exist_ok=True)
os.makedirs(labels_path, exist_ok=True)

# Verified list of public icon URLs
icons_base_url = "https://img.icons8.com/color/96/000000/"
icon_names = [
    "home--v1", "settings--v1", "search--v1", "user-male-circle--v1", "calendar--v1", "camera--v1", "phone--v1", "lock--v1", "cloud--v1",
    "facebook", "twitter", "instagram", "linkedin", "whatsapp", "youtube", "github", "google-logo", "dropbox", "spotify",
    "paypal", "visa", "mastercard", "bitcoin", "alarm", "windows-10", "android-os", "ubuntu", "linux",
    "trash", "download", "upload", "edit", "copy", "paste", "print", "refresh", "save",
    "bookmark", "shopping-cart", "tag", "star", "bell", "document", "briefcase", "key", "map",
    "clock", "gift", "graph", "chat", "network", "shield", "lightning-bolt", "rocket",
    "puzzle", "trophy", "globe", "flag", "compass", "paper-plane", "thumbs-down", "play", "pause",
    "stop", "rewind", "forward", "microphone", "headphones", "speaker", "video",
    "music", "film-reel", "paint-palette", "scissors", "hammer", "wrench", "gear", "car", "bus",
    "train", "bicycle", "motorcycle", "flower", "sun", "moon", "snowflake"
]
icons = [f"{icons_base_url}{name}.png" for name in icon_names]

# Cache downloaded icons to avoid repeated requests
icon_cache = {}

def download_icon(url):
    if url not in icon_cache:
        response = requests.get(url)
        try:
            icon_cache[url] = Image.open(BytesIO(response.content)).convert("RGBA")
        except UnidentifiedImageError:
            print(f"Failed to load image from URL: {url}")
            icon_cache[url] = None
    return icon_cache[url]

def is_overlapping(box1, box2):
    return not (box1[2] <= box2[0] or box1[0] >= box2[2] or box1[3] <= box2[1] or box1[1] >= box2[3])

def generate_synthetic_image(img_idx):
    panel_width, panel_height = 800, 470
    panel_color = tuple(random.randint(150, 255) for _ in range(3))
    panel = Image.new("RGB", (panel_width, panel_height), panel_color)
    draw = ImageDraw.Draw(panel)

    num_panels = random.randint(1, 4)
    annotations = []
    existing_boxes = []

    attempts = 0
    while len(existing_boxes) < num_panels and attempts < 50:
        arrange_vertical = random.choice([True, False])
        num_icons = random.randint(2, 6)
        icon_spacing = 10
        icon_size = random.randint(40, 60)

        if arrange_vertical:
            panel_w = icon_size + 20
            panel_h = num_icons * icon_size + (num_icons - 1) * icon_spacing + 20
        else:
            panel_w = num_icons * icon_size + (num_icons - 1) * icon_spacing + 20
            panel_h = icon_size + 20

        panel_x = random.randint(0, panel_width - panel_w)
        panel_y = random.randint(0, panel_height - panel_h)
        panel_box = [panel_x, panel_y, panel_x + panel_w, panel_y + panel_h]

        if any(is_overlapping(panel_box, eb) for eb in existing_boxes):
            attempts += 1
            continue

        existing_boxes.append(panel_box)
        panel_color_inner = tuple(random.randint(100, 200) for _ in range(3))
        draw.rectangle(panel_box, fill=panel_color_inner)

        x_center = (panel_x + panel_w / 2) / panel_width
        y_center = (panel_y + panel_h / 2) / panel_height
        width = panel_w / panel_width
        height = panel_h / panel_height
        annotations.append(f"1 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

        for i in range(num_icons):
            icon_url = random.choice(icons)
            icon = download_icon(icon_url)

            if icon is None:
                continue

            icon_resized = icon.resize((icon_size, icon_size))

            if arrange_vertical:
                icon_x = panel_x + (panel_w - icon_size) // 2
                icon_y = panel_y + 10 + i * (icon_size + icon_spacing)
            else:
                icon_x = panel_x + 10 + i * (icon_size + icon_spacing)
                icon_y = panel_y + (panel_h - icon_size) // 2

            panel.paste(icon_resized, (icon_x, icon_y), icon_resized)

            x_center_icon = (icon_x + icon_size / 2) / panel_width
            y_center_icon = (icon_y + icon_size / 2) / panel_height
            width_icon = icon_size / panel_width
            height_icon = icon_size / panel_height
            annotations.append(f"0 {x_center_icon:.6f} {y_center_icon:.6f} {width_icon:.6f} {height_icon:.6f}")

        attempts += 1

    img_path = os.path.join(images_path,f'synthetic_{img_idx}.png')
    panel.save(img_path)

    annotation_path = os.path.join(labels_path,f'synthetic_{img_idx}.txt')
    with open(annotation_path, 'w') as f:
        for annotation in annotations:
            f.write(annotation + "\n")

for i in range(50):
    generate_synthetic_image(i)

print("Synthetic UI dataset generated with non-overlapping panels and aligned icons!")

Synthetic UI dataset generated with non-overlapping panels and aligned icons!


# Setup

pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) and check software and hardware.

[![PyPI - Version](https://img.shields.io/pypi/v/ultralytics?logo=pypi&logoColor=white)](https://pypi.org/project/ultralytics/) [![Downloads](https://static.pepy.tech/badge/ultralytics)](https://www.pepy.tech/projects/ultralytics) [![PyPI - Python Version](https://img.shields.io/pypi/pyversions/ultralytics?logo=python&logoColor=gold)](https://pypi.org/project/ultralytics/)

In [15]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.145 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.3/107.7 GB disk)


# Train

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.yaml')  # build a new model from scratch
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)

# Use the model
results = model.train(data='icon.yaml', epochs=5)  # train the model

Ultralytics 8.3.145 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=icon.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, p

train: Scanning /content/datasets/icon/labels/train... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<00:00, 1605.41it/s]

train: New cache created: /content/datasets/icon/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 338.7±202.6 MB/s, size: 20.2 KB)



val: Scanning /content/datasets/icon/labels/val... 51 images, 0 backgrounds, 0 corrupt: 100%|██████████| 51/51 [00:00<00:00, 2100.69it/s]

val: New cache created: /content/datasets/icon/labels/val.cache
Plotting labels to runs/detect/train3/labels.jpg... 


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        2/5         0G      1.176      2.792       1.01        287        640: 100%|██████████| 7/7 [01:34<00:00, 13.51s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:12<00:00,  6.11s/it]

                   all         51        660     0.0241      0.702      0.493      0.393



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        3/5         0G      1.032      1.775     0.9505        877        640:  71%|███████▏  | 5/7 [01:22<00:32, 16.19s/it]

# Validation

In [ ]:
results = model.val()  # evaluate model performance on the validation set